In [14]:
import pandas as pd

# Carregando os dados
df = pd.read_excel('previsao_produto_SCANIA.xlsx')
df['Mês'] = pd.to_datetime(df['Mês'])

# Inicialize um dicionário vazio para as demandas
demandas = {}

# Iterar sobre as linhas do DataFrame e atualizar o dicionário
for index, row in df.iterrows():
    mes_ano = row['Mês']
    previsao = row['Scania']
    
    # Adicionar a previsão ao dicionário usando o mês/ano formatado como chave
    demandas[mes_ano] = previsao

# Definir disponibilidade inicial (pedidos, ordens, estoque, devoluções, etc)
# Criar uma lista com os valores de estoque, ordens, e outros
valores = [50, 30, 5]

# Criar uma série pandas com os valores e os rótulos correspondentes
serie = pd.Series(valores, index=["estoque", "ordens", "outros"])

# Calcular a disponibilidade inicial 
disponibilidade_inicial = serie["estoque"] + serie[["ordens", "outros"]].sum()

# Definir cobertura alvo em meses
cobertura_alvo = 0.33   # POLÍTICA DE ESTOQUE 

# Calcular o estoque alvo para cada período
estoque_alvo = {mes: demanda * cobertura_alvo for mes, demanda in demandas.items()}

# Inicializar o DataFrame do plano de produção
plano_entradas = pd.DataFrame(columns=['Mês', 'Demanda', 'Estoque_Inicial', 'Entradas', 'Estoque_Final', 'Cobertura_Estoque_Dias'])

# Inicializar o DataFrame do balanço de estoque
balanco_estoque = pd.DataFrame(columns=['Mês', 'Entradas', 'Saídas', 'Saldo'])

# Iniciar cálculos a partir de dezembro de 2023
for i, (mes, demanda) in enumerate(demandas.items()):
    # Considerar entradas fixas para dezembro de 2023 e janeiro de 2024
    if i == 0:
        entradas = 1255
    elif i == 1:
        entradas = 1000
    elif i == 2:
        entradas = 1345
    else:
        # Calcular a produção
        entradas = demanda + estoque_alvo[mes] - disponibilidade_inicial

        # Verificar se a produção é negativa
        if entradas < 0:
            entradas = 0

    # Calcular o estoque final
    estoque_final = disponibilidade_inicial + entradas - demanda

    # Calcular a cobertura de estoque em dias
    cobertura_dias = estoque_final / (demanda / 30)

    # Arredondar os valores para zero casas decimais após a vírgula
    entradas = round(entradas, 0)
    estoque_final = round(estoque_final, 0)
    cobertura_dias = round(cobertura_dias, 0)

    # Adicionar os dados ao plano de produção
    plano_entradas.loc[len(plano_entradas)] = [mes, demanda, disponibilidade_inicial, entradas, estoque_final, cobertura_dias]

    # Adicionar os dados ao balanço de estoque
    balanco_estoque.loc[len(balanco_estoque)] = [mes, entradas, demanda, estoque_final]

    # Atualizar o estoque inicial para o próximo mês
    disponibilidade_inicial = estoque_final

# Exibir o DataFrame do plano de produção
print("Plano de Produção:")
print(plano_entradas)

# Exibir o DataFrame do balanço de estoque
print("\nBalanço de Estoque:")
print(balanco_estoque)

Plano de Produção:
          Mês  Demanda  Estoque_Inicial  Entradas  Estoque_Final  \
0  2023-12-01  793.475             85.0    1255.0          547.0   
1  2024-01-01  657.645            547.0    1000.0          889.0   
2  2024-02-01  600.780            889.0    1345.0         1633.0   
3  2024-03-01  703.715           1633.0       0.0          929.0   
4  2024-04-01  644.640            929.0       0.0          284.0   
5  2024-05-01  686.970            284.0     630.0          227.0   
6  2024-06-01  707.965            227.0     715.0          234.0   
7  2024-07-01  747.235            234.0     760.0          247.0   
8  2024-08-01  769.165            247.0     776.0          254.0   
9  2024-09-01  727.515            254.0     714.0          240.0   
10 2024-10-01  744.005            240.0     750.0          246.0   
11 2024-11-01  731.595            246.0     727.0          241.0   
12 2024-12-01  789.820            241.0     809.0          261.0   
13 2025-01-01  659.175       

In [15]:
plano_entradas

,Mês,Demanda,Estoque_Inicial,Entradas,Estoque_Final,Cobertura_Estoque_Dias
0,2023-12-01,793.475,85.0,1255.0,547.0,21.0
1,2024-01-01,657.645,547.0,1000.0,889.0,41.0
2,2024-02-01,600.780,889.0,1345.0,1633.0,82.0
3,2024-03-01,703.715,1633.0,0.0,929.0,40.0
4,2024-04-01,644.640,929.0,0.0,284.0,13.0
5,2024-05-01,686.970,284.0,630.0,227.0,10.0
6,2024-06-01,707.965,227.0,715.0,234.0,10.0
7,2024-07-01,747.235,234.0,760.0,247.0,10.0
8,2024-08-01,769.165,247.0,776.0,254.0,10.0
9,2024-09-01,727.515,254.0,714.0,240.0,10.0


In [16]:
import plotly.graph_objects as go

# Escolhendo as colunas desejadas
colunas_desejadas = ['Mês', 'Demanda', 'Entradas', 'Estoque_Inicial', 'Estoque_Final', 'Cobertura_Estoque_Dias']
plano_entradas_selecionado = plano_entradas[colunas_desejadas]

# Arredondando os valores para 0 casas decimais
plano_entradas_selecionado = plano_entradas_selecionado.round(0)

# Criando a tabela
figura_tabela_plano = go.Figure(data=[go.Table(
    header=dict(values=plano_entradas_selecionado.columns),
    cells=dict(values=plano_entradas_selecionado.transpose().values.tolist())
)])

# Ajuste da largura das colunas
for i, col in enumerate(plano_entradas_selecionado.columns):
    figura_tabela_plano.update_layout(
        {'xaxis'+str(i+1): {'title': col}}
    )

# Layout da tabela
figura_tabela_plano.update_layout(
    title="Plano de Entradas & Cobertura de Estoque",
    margin=dict(l=20, r=20, t=40, b=20),  # Margens
)

# Exibindo a tabela
figura_tabela_plano.show()


In [17]:
import plotly.express as px



# Criar a figura
figura_plano_entradas = px.line(plano_entradas, x='Mês', y=['Demanda','Estoque_Inicial', 'Entradas', 'Estoque_Final'], markers=True)

# Personalizar o layout (títulos, cores, tamanho das fontes, legendas)
figura_plano_entradas.update_layout(
    title='Plano de Entradas e Estoque',
    xaxis_title='Mês',
    yaxis_title='Quantidade',
    legend_title='Categorias',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

# Adicionar anotação sobre a cobertura de estoque
figura_plano_entradas.add_annotation(
    xref='paper', yref='paper',
    x=0.95, y=0.9,
    text='Cobertura de Estoque: 10 dias',
    showarrow=False,
    font=dict(
        family="Arial, sans-serif",
        size=14,
        color="Black"
    )
)

# Mostrar a figura
figura_plano_entradas.show()


In [27]:
# Selecionar colunas "Mês" e "Margem de Contribuição"
cobertura_dias = plano_entradas.loc[:, ["Mês", "Cobertura_Estoque_Dias"]]

# Exibir o DataFrame resultante
print(cobertura_dias)

          Mês  Cobertura_Estoque_Dias
0  2023-12-01                    21.0
1  2024-01-01                    41.0
2  2024-02-01                    82.0
3  2024-03-01                    40.0
4  2024-04-01                    13.0
5  2024-05-01                    10.0
6  2024-06-01                    10.0
7  2024-07-01                    10.0
8  2024-08-01                    10.0
9  2024-09-01                    10.0
10 2024-10-01                    10.0
11 2024-11-01                    10.0
12 2024-12-01                    10.0
13 2025-01-01                    10.0
14 2025-02-01                    10.0
15 2025-03-01                    10.0
16 2025-04-01                    10.0
17 2025-05-01                    10.0
18 2025-06-01                    10.0
19 2025-07-01                    10.0
20 2025-08-01                    10.0
21 2025-09-01                    10.0
22 2025-10-01                    10.0
23 2025-11-01                    10.0
24 2025-12-01                    10.0
25 2026-01-0

In [28]:
import plotly.express as px
import pandas as pd

# Supondo que 'Mês' seja a coluna de datas no seu DataFrame
cobertura_dias['Mês'] = pd.to_datetime(cobertura_dias['Mês'])

# Criar a figura com Plotly Express
figura_cobertura_dias = px.line(cobertura_dias, x='Mês', y='Cobertura_Estoque_Dias', markers=True)

# Adicionar uma linha horizontal para o KPI de 15 dias
figura_cobertura_dias.add_shape(
    type='line',
    x0=df['Mês'].min(),
    x1=df['Mês'].max(),
    y0=10,
    y1=10,
    line=dict(color='red', dash='dash'),
    name='KPI 10 dias'
)

# Adicionar anotação com a legenda do KPI
figura_cobertura_dias.add_annotation(
    x=df['Mês'].mean(),
    y=10,
    text='KPI 10 dias',
    showarrow=True,
    arrowhead=2,
    arrowcolor='red',
    ax=0,
    ay=-40
)

# Personalizar o layout (títulos, cores, tamanho das fontes, legendas)
figura_cobertura_dias.update_layout(
    title='Cobertura de Estoque',
    xaxis_title='Mês',
    yaxis_title='Dias',
    legend_title='Cobertura de Estoque',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

# Mostrar a figura
figura_cobertura_dias.show()

# PLANEJAMENTO FINANCEIRO (RECEITA, CUSTO DO PRODUTO VENDIDO E MARGEM DE CONTRIBUIÇÃO)¶¶

In [18]:
# Seu DataFrame com as previsões, preço e cpv
# Carregando os dados
financeiro = pd.read_excel('FINANCEIRO_CAMINHÕES_Scania.xlsx')
#Alterando a coluna "ano" de "int" para "str"
financeiro['Mês'] = financeiro['Mês'].astype(str)
financeiro

,Mês,Scania,Preço_médio,Custo_Produto_Vendido
0,2023-12-01,793.475,658765.80,483021.0000
1,2024-01-01,657.645,658765.80,484788.1500
2,2024-02-01,600.780,639523.50,483021.0000
3,2024-03-01,703.715,633864.00,517068.0900
4,2024-04-01,644.640,645183.00,492445.8000
5,2024-05-01,686.970,673480.50,494212.9500
6,2024-06-01,707.965,672348.60,519542.1000
7,2024-07-01,747.235,662161.50,517686.5925
8,2024-08-01,769.165,656502.00,516202.1865
9,2024-09-01,727.515,658765.80,513975.5775


In [19]:
# Calcular Receita Total
financeiro['Receita Total'] = financeiro['Scania'] * financeiro['Preço_médio']

# Calcular Custo Total
financeiro['Custo Total'] = financeiro['Scania'] * financeiro['Custo_Produto_Vendido']

# Calcular Lucro
financeiro['Lucro'] = financeiro['Receita Total'] - financeiro['Custo Total']

# Calcular Margem de Contribuição
financeiro['Margem de Contribuição'] = (financeiro['Lucro'] / financeiro['Receita Total']) * 100

# Imprimir os resultados mês a mês
print(financeiro[['Mês', 'Receita Total', 'Custo Total', 'Lucro', 'Margem de Contribuição']])


           Mês  Receita Total   Custo Total         Lucro  \
0   2023-12-01   5.227142e+08  3.832651e+08  1.394491e+08   
1   2024-01-01   4.332340e+08  3.188185e+08  1.144155e+08   
2   2024-02-01   3.842129e+08  2.901894e+08  9.402357e+07   
3   2024-03-01   4.460596e+08  3.638686e+08  8.219103e+07   
4   2024-04-01   4.159108e+08  3.174503e+08  9.846051e+07   
5   2024-05-01   4.626609e+08  3.395095e+08  1.231514e+08   
6   2024-06-01   4.759993e+08  3.678176e+08  1.081817e+08   
7   2024-07-01   4.947902e+08  3.868335e+08  1.079567e+08   
8   2024-08-01   5.049584e+08  3.970447e+08  1.079137e+08   
9   2024-09-01   4.792620e+08  3.739249e+08  1.053371e+08   
10  2024-10-01   4.926515e+08  3.641909e+08  1.284606e+08   
11  2024-11-01   4.844340e+08  3.585471e+08  1.258869e+08   
12  2024-12-01   5.220944e+08  3.889435e+08  1.331509e+08   
13  2025-01-01   4.364803e+08  3.249962e+08  1.114841e+08   
14  2025-02-01   4.074759e+08  2.990137e+08  1.084622e+08   
15  2025-03-01   4.79166

In [20]:
financeiro

,Mês,Scania,Preço_médio,Custo_Produto_Vendido,Receita Total,Custo Total,Lucro,Margem de Contribuição
0,2023-12-01,793.475,658765.80,483021.0000,5.227142e+08,3.832651e+08,1.394491e+08,26.677888
1,2024-01-01,657.645,658765.80,484788.1500,4.332340e+08,3.188185e+08,1.144155e+08,26.409636
2,2024-02-01,600.780,639523.50,483021.0000,3.842129e+08,2.901894e+08,9.402357e+07,24.471736
3,2024-03-01,703.715,633864.00,517068.0900,4.460596e+08,3.638686e+08,8.219103e+07,18.426020
4,2024-04-01,644.640,645183.00,492445.8000,4.159108e+08,3.174503e+08,9.846051e+07,23.673469
5,2024-05-01,686.970,673480.50,494212.9500,4.626609e+08,3.395095e+08,1.231514e+08,26.618076
6,2024-06-01,707.965,672348.60,519542.1000,4.759993e+08,3.678176e+08,1.081817e+08,22.727273
7,2024-07-01,747.235,662161.50,517686.5925,4.947902e+08,3.868335e+08,1.079567e+08,21.818681
8,2024-08-01,769.165,656502.00,516202.1865,5.049584e+08,3.970447e+08,1.079137e+08,21.370813
9,2024-09-01,727.515,658765.80,513975.5775,4.792620e+08,3.739249e+08,1.053371e+08,21.979013


In [21]:
import plotly.express as px

# Criando o gráfico com Plotly Express
fig_financeiro = px.line(financeiro, x="Mês", y=['Preço_médio','Custo_Produto_Vendido'], title='Preços & Custos')

# Alterando a cor da linha para 'Receita Total'
fig_financeiro.update_traces(line=dict(color='red'), selector=dict(name='Preço_médio'))

# Alterando a cor da linha para 'Custo Total'
fig_financeiro.update_traces(line=dict(color='green'), selector=dict(name='Custo_Produto_Vendido'))


# Alterando o título do eixo x
fig_financeiro.update_layout(xaxis_title='Meses')

# Alterando o título do eixo y
fig_financeiro.update_layout(yaxis_title='Valores')

# Alterando o título do gráfico
fig_financeiro.update_layout(title='Preços & Custos')

# Alterando a fonte e o tamanho de letra do título
fig_financeiro.update_layout(title_font=dict(family='Arial', size=18))

# Alterando o estilo da linha para trace 'Receita Total'
fig_financeiro.update_traces(line=dict(dash='dash'), selector=dict(name='Preço_médio'))

# Exibindo o gráfico
fig_financeiro.show()


In [22]:
import plotly.express as px

# Criando o gráfico com Plotly Express
fig_financeiro1 = px.line(financeiro, x="Mês", y=['Receita Total','Custo Total', 'Lucro'], title='Análise Financeira')

# Alterando a cor da linha para 'Receita Total'
fig_financeiro1.update_traces(line=dict(color='red'), selector=dict(name='Receita Total'))

# Alterando a cor da linha para 'Custo Total'
fig_financeiro1.update_traces(line=dict(color='green'), selector=dict(name='Custo Total'))

# Alterando a cor da linha para 'Custo Total'
fig_financeiro1.update_traces(line=dict(color='blue'), selector=dict(name='Lucro'))


# Alterando o título do eixo x
fig_financeiro1.update_layout(xaxis_title='Meses')

# Alterando o título do eixo y
fig_financeiro1.update_layout(yaxis_title='Valores')

# Alterando o título do gráfico
fig_financeiro1.update_layout(title='Análise Financeira')

# Alterando a fonte e o tamanho de letra do título
fig_financeiro1.update_layout(title_font=dict(family='Arial', size=18))

# Alterando o estilo da linha para trace 'Receita Total'
fig_financeiro1.update_traces(line=dict(dash='dash'), selector=dict(name='Receita Total'))

# Exibindo o gráfico
fig_financeiro1.show()


In [23]:
# Selecionar colunas "Mês" e "Margem de Contribuição"
financeiro_margem = financeiro.loc[:, ["Mês", "Margem de Contribuição"]]

# Exibir o DataFrame resultante
print(financeiro_margem)


           Mês  Margem de Contribuição
0   2023-12-01               26.677888
1   2024-01-01               26.409636
2   2024-02-01               24.471736
3   2024-03-01               18.426020
4   2024-04-01               23.673469
5   2024-05-01               26.618076
6   2024-06-01               22.727273
7   2024-07-01               21.818681
8   2024-08-01               21.370813
9   2024-09-01               21.979013
10  2024-10-01               26.075353
11  2024-11-01               25.986395
12  2024-12-01               25.503215
13  2025-01-01               25.541601
14  2025-02-01               26.618076
15  2025-03-01               23.754153
16  2025-04-01               26.884804
17  2025-05-01               27.039976
18  2025-06-01               26.329050
19  2025-07-01               25.185915
20  2025-08-01               21.227364
21  2025-09-01               21.810385
22  2025-10-01               21.400422
23  2025-11-01               21.529274
24  2025-12-01           

In [24]:
financeiro_margem

,Mês,Margem de Contribuição
0,2023-12-01,26.677888
1,2024-01-01,26.409636
2,2024-02-01,24.471736
3,2024-03-01,18.426020
4,2024-04-01,23.673469
5,2024-05-01,26.618076
6,2024-06-01,22.727273
7,2024-07-01,21.818681
8,2024-08-01,21.370813
9,2024-09-01,21.979013


In [25]:
import plotly.express as px
import pandas as pd

# Supondo que 'Mês' seja a coluna de datas no seu DataFrame
df['Mês'] = pd.to_datetime(df['Mês'])

# Criar a figura com Plotly Express
figura_plano_financeiro = px.line(financeiro_margem, x='Mês', y='Margem de Contribuição', markers=True)

# Adicionar uma linha horizontal para o KPI de 25%
figura_plano_financeiro.add_shape(
    type='line',
    x0=df['Mês'].min(),
    x1=df['Mês'].max(),
    y0=25,
    y1=25,
    line=dict(color='red', dash='dash'),
    name='KPI 25%'
)

# Adicionar anotação com a legenda do KPI
figura_plano_financeiro.add_annotation(
    x=df['Mês'].mean(),
    y=25,
    text='KPI 25%',
    showarrow=True,
    arrowhead=2,
    arrowcolor='red',
    ax=0,
    ay=-40
)

# Personalizar o layout (títulos, cores, tamanho das fontes, legendas)
figura_plano_financeiro.update_layout(
    title='Margem Contribuição',
    xaxis_title='Mês',
    yaxis_title='%',
    legend_title='Margem de Contribuição',
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="RebeccaPurple"
    )
)

# Mostrar a figura
figura_plano_financeiro.show()


In [26]:
# Importando as bibliotecas necessárias
from datetime import datetime
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc


# Inicializando o aplicativo Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.COSMO])

# Layout do aplicativo
app.layout = dbc.Container(
    [
        dbc.Tabs(
            [
                dbc.Tab(
                    label='Introdução',
                    tab_id='tab-1',
                    children=[
                        dbc.Row(
                            [
                                dbc.Col(
                                    html.Img(
                                        src='https://transportemoderno.com.br/wp-content/uploads/2020/07/Scania-edi%C3%A7%C3%A3o-especial.png',
                                        style={'width': '60%'}
                                    ),
                                    width=6
                                ),
                                dbc.Col(
                                    html.Img(
                                        src='https://th.bing.com/th/id/OIP.ab6rZRsapELFt_LyExZtwwHaDx?rs=1&pid=ImgDetMain',
                                        style={'width': '90%'}
                                    ),
                                    width=6
                                ),
                            ]
                        ),
                        html.P(),
                        dbc.Row(
                            [
                                dbc.Col(
                                    [
                                        html.H1('Plano de Entradas & Financeiro - Produto: Scania'),
                                        html.P(),
                                        dbc.Badge('Sales & Operation Planning (S&OP)', color='success',
                                                  className='ms-1', style={'font-size': '30px'}),
                                    ]
                                ),
                            ]
                        ),
                        dbc.Card(
                            dbc.CardBody(
                                [
                                    html.H3('Data e Hora', className='card-title'),
                                    html.P(id='current-time', className='card-text'),
                                    dcc.Interval(
                                        id='interval-component',
                                        interval=1000,  # intervalos a cada 1000 milisegundos (1 segundo)
                                        n_intervals=0
                                    ),
                                ]
                            ),
                            style={'padding': '10px'}
                        ),
                    ]
                ),
                
                dbc.Tab(
                    label='Plano de Produção & Cobertura de Estoque',
                    tab_id='tab-2',
                    children=[
                        dbc.Col(
                            [
                                html.H1('Plano de Produção & Cobertura de Estoque - Com Time Freeze Lead Time = 3 meses'), 
                                html.P(),
                            ]
                        ),
                        dcc.Markdown(
                            """
                            *O plano de produção é a etapa do S&OP que define a quantidade e o momento da produção de 
                            cada produto, considerando a demanda prevista, a capacidade disponível e os níveis de estoque 
                            desejados. 
                            
                            *A cobertura de estoque é o indicador que mede quantos meses ou dias de demanda podem ser atendidos
                            com o estoque atual, sendo um importante instrumento para avaliar o equilíbrio entre oferta e demanda 
                            no S&OP.
                            """, 
                            style={'font-family': 'Georgia, serif', 'font-size':'25', 'font-weight': 'bold'}
                        ), 
                        html.P(),
                        
                        dbc.Col(
                            [
                                html.H2('Plano de Entradas & Cobertura de Estoque'), 
                                html.P(),
                            ]
                        ),
                        
                        dbc.Col(dcc.Graph(id='tabela-plano', figure=figura_tabela_plano), width=12), html.P(),
                        dbc.Col(dcc.Graph(id='line-plano', figure=figura_plano_entradas), width=12), html.P(),
                        
                        # Adicione aqui os componentes do gráfico e tabela que faltam
                        # Por exemplo, dcc.Graph(id='tabela-plano', figure=figura_tabela_plano),
                        # dcc.Graph(id='line-plano', figure=figura_plano_entradas),
                        
                        
                    ]
                ),
                
                dbc.Tab(
                label='Plano Financeiro',
                tab_id='tab-3',
                children=[
                    
                     dbc.Col(
                         
                       [
                           html.H1('Plano Financeiro'), 
                           html.P(),
                       ]
                         
                       ),
                    
                    dcc.Markdown(
                    """
                    *A análise financeira no S&OP é uma ferramenta importante para avaliar 
                    o impacto das decisões de planejamento na rentabilidade e no fluxo de caixa da empresa. 
                    
                    *Ela permite comparar diferentes cenários e identificar as melhores alternativas para atender
                    às demandas dos clientes e aos objetivos estratégicos. 
                    
                    *A análise financeira no S&OP envolve a estimativa dos preços, custos, receitas, margens,  
                    associados aos planos de produção, vendas, estoques e compras.                    
                    
                    """,
                    style={'font-family': 'Georgia, serif', 'font-size':'25', 'font-weight': 'bold'}
                    ),
                    html.P(),
                    
                    dbc.Col(
                        [
                           
                            html.H2('Preços & Custos'), html.P(),
                            
                        ]
                    ),
                    
                    dbc.Col(dcc.Graph(id='line-preçoscustos', figure=fig_financeiro), width=12), 
                    html.P(),
                    
             
                
                dbc.Col(
                    [
                       
                        html.H2('Receitas, Custos e Lucros'), html.P(),
                        
                    ]
                ),
                
                dbc.Col(dcc.Graph(id='line-receitas', figure=fig_financeiro1), width=12), 
                html.P(),
                
                dbc.Col(
                    [
                       
                        html.H2('Margem de Contribuição'), html.P(),
                        
                        
                    ]
                ),
                
                dbc.Col(dcc.Graph(id='line-margem', figure=figura_plano_financeiro), width=12),
                html.P(),
                
               ]
            ),  
                     
            ]
        ),
    ],
    fluid=True
)

# Atualização da data e hora
@app.callback(
    Output("current-time", "children"),
    Input("interval-component", "n_intervals"),
)
def update_time(n):
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"Data e Hora Atuais: {current_time}"

# Executando o aplicativo Dash
if __name__ == "__main__":
    app.run_server(debug=True, port=8027)
